<a href="https://colab.research.google.com/github/EgonFerri/codemotion-challenge/blob/main/codemotion_challenge_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd
import polars as pl
import requests
from typing import Tuple, Sequence, List, Dict


response = requests.get('https://raw.githubusercontent.com/EgonFerri/codemotion-challenge/refs/heads/main/attacking_types_chart.csv')
with open('attacking_types_chart.csv', "wb") as file:
  file.write(response.content)

response = requests.get('https://raw.githubusercontent.com/EgonFerri/codemotion-challenge/refs/heads/main/pokemon.csv')
with open('pokemon.csv', "wb") as file:
    file.write(response.content)

# Challenge: PokéBattle Champion – Type Multipliers Edition Background


You’re a Codemotion trainer with a team of 6 Pokémon (all from the first two generations), and you’re about to face a single opponent (a real Gen 1 or Gen 2 Pokémon).
Your job is to choose the best Pokémon to battle the opponent by computing a score for each team member based on its stats and type interactions.

# The Challenge

### Your Function Must Do the Following

Implement the function with the following signature:

```python
def choose_best_pokemon(team: dict[str, int], opponent: dict[str, int]) -> str:
    """
    team: a dictionary mapping Pokémon names (str) to desired levels (int)
          e.g., {"pikachu": 32, "charmander": 30, ...}
    opponent: a dictionary with keys "name" (str) and "level" (int)
          e.g., {"name": "geodude", "level": 20}
    
    Returns:
      The name (str) of the team Pokémon with the highest computed score.
    """
```

Your function must perform the following steps:

1. **Data Loading:**  
   - Use a helper function (e.g., `read_data(file1, file2)`) to load the CSV files:
     - `pokemon.csv` containing each Pokémon’s record (including `"Attack"`, `"Defense"`, `"Total"`, `"Type 1"`, and `"Type 2"`).
     - `attacking_types_chart.csv` to build a lookup table (e.g., `types_table`) for attacking multipliers.
   - Normalize Pokémon names and type names (e.g., convert to lowercase) for correct lookups.

2. **Retrieve Opponent Data:**  
   - Using a helper function (e.g., `get_pokemon_data(pokemon_data, pokemon_name)`), obtain the opponent’s data from `pokemon.csv`.  
   - This function should return a tuple:  
     ```python
     (opponent_types, opponent_Attack, opponent_Defense)
     ```  
     where `opponent_types` is a list containing the opponent’s `"Type 1"` and (if nonempty) `"Type 2"`.
   - Use the opponent’s level as provided in the `opponent` dictionary (do not compute it from the `"Total"` stat).

3. **Iterate Over the Team:**  
   For each Pokémon in the `team` dictionary:
   - Retrieve the team Pokémon’s data (its types, Attack, and Defense) via `get_pokemon_data`.
   - Compute the attacking bonus when the team Pokémon attacks the opponent by calling a helper function, for example:  
     ```python
     bonus1 = calculate_attacking_bonus(types_table, team_types, opponent_types)
     ```
   - Compute the attacking bonus for the opponent attacking the team Pokémon by calling:  
     ```python
     bonus2 = calculate_attacking_bonus(types_table, opponent_types, team_types)
     ```
   - Compute the score using a helper function (e.g., `calculate_score`) with the formula:
     ```python
     score = (Attack * bonus1 - opponent_Defense) + \
             (Defense - opponent_Attack * bonus2) + \
             5 * (team_level - opponent_level)
     ```
     Here, `team_level` is the level provided for that Pokémon in the team dictionary.

4. **Select the Best Pokémon:**  
   - Keep track of the Pokémon with the highest computed score.
   - Optionally, print a message indicating the opponent’s name and the best choice.

5. **Return:**  
   - Return the name (key from the team dictionary) of the Pokémon with the highest score.
   - If the team is empty, return `None`.

---

Implement your solution following this outline. Partial credit may be given for correctly implementing individual helper functions (such as data loading, computing the attacking bonus, or calculating the score) even if the full solution is not completed.

# Your solution


## Read data

#### Your function

In [22]:
def read_data(file1: str, file2: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Reads two CSV files into pandas DataFrames.

    Parameters:
      - file1: Path to 'pokemon.csv'.
      - file2: Path to 'attacking_types_chart.csv'.

    Returns:
      A tuple (pokemon_data, types_data) where:
        - pokemon_data is a DataFrame containing the Pokémon records.
        - types_data is a DataFrame containing the attacking types chart.
    """
    pokemon_data = pd.read_csv(file1, index_col=0)
    types_data = pd.read_csv(file2, index_col=0)
    return pokemon_data, types_data

pokemons, types_table = read_data('pokemon.csv', 'attacking_types_chart.csv')

## Calculate score (1 pt)


#### Your function

In [25]:
def calculate_score(att1: int, def1: int, att2: int, def2: int,
                    lev1: int, lev2: int, attbonus1: float, attbonus2: float) -> float:
    """
    Computes the score for a matchup between a team Pokémon and the opponent.

    Formula:
      score = (Attack * attacking_bonus - opponent_Defense) +
              (Defense - opponent_Attack * attacking_bonus2) +
              5 * (team_level - opponent_level)

    Parameters:
      - att1: Team Pokémon's Attack.
      - def1: Team Pokémon's Defense.
      - att2: Opponent's Attack.
      - def2: Opponent's Defense.
      - lev1: Level of the team Pokémon.
      - lev2: Level of the opponent.
      - attbonus1: Attacking bonus multiplier when team Pokémon attacks.
      - attbonus2: Attacking bonus multiplier when opponent attacks.

    Returns:
      The computed score as a float.
    """
    a = (att1 * attbonus1) - def2
    b = def1 - (att2 * attbonus2)
    c = 5 * (lev1 - lev2)
    return a + b + c

#### Tests

In [47]:
def test_calculate_score_case1():
    # Test with simple numbers.
    # Team Pokémon: Attack=50, Defense=40, Level=10, attbonus1=2.0
    # Opponent: Attack=20, Defense=30, Level=8, attbonus2=1.0
    # Expected:
    #   a = (50*2.0 - 30) = 100 - 30 = 70
    #   b = (40 - 20*1.0) = 40 - 20 = 20
    #   c = 5*(10-8) = 5*2 = 10
    # Total score = 70 + 20 + 10 = 100
    score = calculate_score(50, 40, 20, 30, 10, 8, 2.0, 1.0)
    assert score == 100, f"Expected 100, got {score}"

def test_calculate_score_case2():
    # Another test with different parameters.
    # Team Pokémon: Attack=60, Defense=30, Level=15, attbonus1=1.5
    # Opponent: Attack=40, Defense=40, Level=12, attbonus2=0.5
    # Expected:
    #   a = (60*1.5 - 40) = 90 - 40 = 50
    #   b = (30 - 40*0.5) = 30 - 20 = 10
    #   c = 5*(15-12) = 5*3 = 15
    # Total score = 50 + 10 + 15 = 75
    score = calculate_score(60, 30, 40, 40, 15, 12, 1.5, 0.5)
    expected = 75
    assert score == expected, f"Expected {expected}, got {score}"

def test_calculate_score_negative():
    # Test where differences lead to negative contributions.
    # Team Pokémon: Attack=30, Defense=50, Level=10, attbonus1=1.2
    # Opponent: Attack=60, Defense=70, Level=12, attbonus2=1.2
    # Expected:
    #   a = (30*1.2 - 70) = 36 - 70 = -34
    #   b = (50 - 60*1.2) = 50 - 72 = -22
    #   c = 5*(10-12) = -10
    # Total score = -34 + (-22) + (-10) = -66
    score = calculate_score(30, 50, 60, 70, 10, 12, 1.2, 1.2)
    expected = -66
    assert score == expected, f"Expected {expected}, got {score}"

if __name__ == "__main__":
    test_calculate_score_case1()
    test_calculate_score_case2()
    test_calculate_score_negative()
    print("All calculate_score tests passed.")

All calculate_score tests passed.


## Calculate attacking bonus (2pt)

#### Your function

In [49]:
def calculate_attacking_bonus(types_table: pd.DataFrame,
                              types1: Sequence[str] = (),
                              types2: Sequence[str] = ()) -> float:
    """
    Computes the attacking bonus as the product of multipliers for every pairing
    of an attacker's type (from types1) and a defender's type (from types2).

    Parameters:
      - types_table: A DataFrame representing the attacking types chart.
      - types1: A sequence of attacker type names (e.g., ('grass', 'poison')).
      - types2: A sequence of defender type names (e.g., ('rock', 'ground')).

    Returns:
      The product of all corresponding multipliers as a float.
    """
    bonus = 1.0
    for attacker_type in types1:
        for defender_type in types2:
            bonus *= types_table.loc[attacker_type, defender_type]
    return bonus

#### Tests

In [48]:
def test_bonus_single():
    # Test: Attacker 'Grass' vs Defender 'Fire'
    # Expected: Look up row "Grass", column "Fire" → 0.5
    bonus = calculate_attacking_bonus(types_table, ['Grass'], ['Fire'])
    expected = 0.5
    assert abs(bonus - expected) < 1e-6, f"Expected {expected}, got {bonus}"

def test_bonus_multiple():
    # Test: Attacker types ['Fire', 'Fighting'] vs Defender types ['Rock', 'Water']
    # For attacker 'Fire' vs 'Rock': row "Fire", column "Rock" → 0.5
    # For attacker 'Fire' vs 'Water': row "Fire", column "Water" → 0.5
    # For attacker 'Fighting' vs 'Rock': row "Fighting", column "Rock" → 2
    # For attacker 'Fighting' vs 'Water': row "Fighting", column "Water" → 1
    # Total product = 0.5 * 0.5 * 2 * 1 = 0.5
    bonus = calculate_attacking_bonus(types_table, ['Fire', 'Fighting'], ['Rock', 'Water'])
    expected = 0.5
    assert abs(bonus - expected) < 1e-6, f"Expected {expected}, got {bonus}"

def test_bonus_empty_attacker():
    # Test: Empty attacker types should yield a product of 1.0.
    bonus = calculate_attacking_bonus(types_table, [], ['Rock', 'Water'])
    expected = 1.0
    assert abs(bonus - expected) < 1e-6, f"Expected {expected}, got {bonus}"

def test_bonus_empty_defender():
    # Test: Empty defender types should yield a product of 1.0.
    bonus = calculate_attacking_bonus(types_table, ['Fire'], [])
    expected = 1.0
    assert abs(bonus - expected) < 1e-6, f"Expected {expected}, got {bonus}"

def test_bonus_both_empty():
    # Test: Both attacker and defender types empty should yield 1.0.
    bonus = calculate_attacking_bonus(types_table, [], [])
    expected = 1.0
    assert abs(bonus - expected) < 1e-6, f"Expected {expected}, got {bonus}"

if __name__ == "__main__":
    test_bonus_single()
    test_bonus_multiple()
    test_bonus_empty_attacker()
    test_bonus_empty_defender()
    test_bonus_both_empty()
    print("All calculate_attacking_bonus tests passed.")

All calculate_attacking_bonus tests passed.


## Get pokemon data (2pt)

#### Your function

In [39]:

def get_pokemon_data(pokemon_data: pd.DataFrame, pokemon_name: str) -> Tuple[List[str], int, int]:
    """
    Retrieves a Pokémon's type(s), Attack, and Defense from the provided DataFrame.

    Parameters:
      - pokemon_data: A DataFrame containing the Pokémon records from 'pokemon.csv'.
      - pokemon_name: The name of the Pokémon (case-insensitive).

    Returns:
      A tuple (types, Attack, Defense), where:
        - types is a list of the Pokémon's types (e.g., ['fire'] or ['grass', 'poison']).
        - Attack is the Pokémon's Attack stat (int).
        - Defense is the Pokémon's Defense stat (int).
    """
    # Normalize the Pokémon name (capitalize as needed to match CSV)
    pkmn = pokemon_name.capitalize()
    row = pokemon_data[pokemon_data['Name'] == pkmn].reset_index(drop=True).to_dict()
    types = [row['Type 1'][0]]
    if isinstance(row['Type 2'][0], str) and row['Type 2'][0].strip() != "":
        types.append(row['Type 2'][0])
    return types, row['Attack'][0], row['Defense'][0]

#### Tests

In [54]:
df = pokemons

def test_get_pokemon_data_bulbasaur():
    types, attack, defense = get_pokemon_data(df, "bulbasaur")
    # Expected: Bulbasaur has two types: "Grass" and "Poison"; Attack 49, Defense 49.
    assert types == ["Grass", "Poison"], f"Expected ['Grass', 'Poison'], got {types}"
    assert attack == 49, f"Expected Attack 49, got {attack}"
    assert defense == 49, f"Expected Defense 49, got {defense}"

def test_get_pokemon_data_charmander():
    types, attack, defense = get_pokemon_data(df, "charmander")
    # Expected: Charmander has one type: "Fire"; Attack 52, Defense 43.
    assert types == ["Fire"], f"Expected ['Fire'], got {types}"
    assert attack == 52, f"Expected Attack 52, got {attack}"
    assert defense == 43, f"Expected Defense 43, got {defense}"

def test_get_pokemon_data_squirtle():
    types, attack, defense = get_pokemon_data(df, "squirtle")
    # Expected: Squirtle has one type: "Water"; Attack 48, Defense 65.
    assert types == ["Water"], f"Expected ['Water'], got {types}"
    assert attack == 48, f"Expected Attack 48, got {attack}"
    assert defense == 65, f"Expected Defense 65, got {defense}"

if __name__ == "__main__":
    test_get_pokemon_data_bulbasaur()
    test_get_pokemon_data_charmander()
    test_get_pokemon_data_squirtle()
    print("All get_pokemon_data tests passed!")

All get_pokemon_data tests passed!


## Choose best pokemon (5 pt)

#### Your function

In [68]:
def choose_best_pokemon(team: Dict[str, int], opponent: Dict[str, int]) -> str:
    """
    Selects the best Pokémon from the team to face the opponent based on computed scores.

    Parameters:
      - team: A dictionary mapping Pokémon names (str) to desired levels (int).
              Example: {"pikachu": 32, "charmander": 30, ...}
      - opponent: A dictionary with keys "name" (str) and "level" (int).
                  Example: {"name": "geodude", "level": 20}

    Returns:
      The name (str) of the team Pokémon with the highest computed score.
    """
    opp_level = opponent['level']
    opp_name = opponent['name']
    # Retrieve opponent's data
    opp_types, opp_att, opp_defe = get_pokemon_data(pokemons, opp_name)

    best_score = -1e6
    best_pokemon = None

    for pokemon, level in team.items():
        tipi, att, defe = get_pokemon_data(pokemons, pokemon)
        bonus1 = calculate_attacking_bonus(types_table, tipi, opp_types)
        bonus2 = calculate_attacking_bonus(types_table, opp_types, tipi)
        score = calculate_score(att, defe, opp_att, opp_defe, level, opp_level, bonus1, bonus2)
        if score > best_score:
            best_score = score
            best_pokemon = pokemon

    return best_pokemon

#### Tests

In [72]:
def test1():
    # Team: Gen 1 Pokémon; Opponent: Geodude (Rock/Ground)
    team = {
        "pikachu": 32,
        "charmander": 30,
        "bulbasaur": 33,
        "squirtle": 31,
        "eevee": 35,
        "pidgey": 28
    }
    opponent = {"name": "geodude", "level": 20}
    result = choose_best_pokemon(team, opponent)
    print(f'Choosing {result} against {opponent["name"]}:')
    assert result == "squirtle", f"Expected 'squirtle', got {result}"

def test2():
    # Team: Gen 1 Pokémon; Opponent: Onix (Rock/Ground)
    team = {
        "pikachu": 32,
        "charmander": 30,
        "bulbasaur": 33,
        "squirtle": 31,
        "eevee": 35,
        "pidgey": 28
    }
    opponent = {"name": "onix", "level": 22}
    result = choose_best_pokemon(team, opponent)
    print(f'Choosing {result} against {opponent["name"]}:')
    assert result == "squirtle", f"Expected 'squirtle', got {result}"

def test3():
    # Team: Gen 2 Pokémon; Opponent: Umbreon (Dark)
    team = {
        "chikorita": 30,
        "cyndaquil": 30,
        "totodile": 30,
        "pichu": 25,
        "togepi": 26,
        "mareep": 27
    }
    opponent = {"name": "umbreon", "level": 30}
    result = choose_best_pokemon(team, opponent)
    print(f'Choosing {result} against {opponent["name"]}:')
    assert result == "totodile", f"Expected 'totodile', got {result}"

def test4():
    # Team: Mixed Gen; Opponent: Murkrow (Dark/Flying)
    team = {
        "pikachu": 32,
        "bulbasaur": 33,
        "squirtle": 31,
        "chikorita": 30,
        "cyndaquil": 30,
        "totodile": 30
    }
    opponent = {"name": "murkrow", "level": 25}
    result = choose_best_pokemon(team, opponent)
    print(f'Choosing {result} against {opponent["name"]}:')
    assert result == "pikachu", f"Expected 'pikachu', got {result}"

def test5():
    # Team: Gen 1 Pokémon; Opponent: Gyarados (Water/Flying)
    team = {
        "machop": 29,
        "gastly": 30,
        "onix": 27,
        "hitmonlee": 31,
        "jigglypuff": 25,
        "snorlax": 35
    }
    opponent = {"name": "gyarados", "level": 35}
    result = choose_best_pokemon(team, opponent)
    print(f'Choosing {result} against {opponent["name"]}:')
    assert result == "snorlax", f"Expected 'snorlax', got {result}"

def test6():
    # Team: Early Gen 1 Pokémon; Opponent: Alakazam (Psychic)
    team = {
        "pidgey": 20,
        "rattata": 20,
        "ekans": 22,
        "sandshrew": 100,
        "lugia": 23,
        "clefairy": 24
    }
    opponent = {"name": "alakazam", "level": 28}
    result = choose_best_pokemon(team, opponent)
    print(f'Choosing {result} against {opponent["name"]}:')
    assert result == "sandshrew", f"Expected 'sandshrew', got {result}"

def test7():
    # Team: Gen 2 Pokémon; Opponent: Feraligatr (Water)
    team = {
        "chikorita": 30,
        "cyndaquil": 30,
        "totodile": 30,
        "bellossom": 34,
        "hoothoot": 22,
        "ledyba": 20
    }
    opponent = {"name": "feraligatr", "level": 32}
    result = choose_best_pokemon(team, opponent)
    print(f'Choosing {result} against {opponent["name"]}:')
    assert result == "bellossom", f"Expected 'bellossom', got {result}"

def test8():
    # Team: Gen 1 Pokémon; Opponent: Dragonite (Dragon/Flying)
    team = {
        "pikachu": 32,
        "charmander": 30,
        "bulbasaur": 33,
        "squirtle": 31,
        "jigglypuff": 25,
        "meowth": 28
    }
    opponent = {"name": "dragonite", "level": 40}
    result = choose_best_pokemon(team, opponent)
    print(f'Choosing {result} against {opponent["name"]}:')
    assert result == "jigglypuff", f"Expected 'jigglypuff', got {result}"

if __name__ == "__main__":
    tests = [test1, test2, test3, test4, test5, test6, test7, test8]
    for i, test in enumerate(tests, 1):
        try:
            test()
            print(f"Test {i} passed, it was the best choice for the battle")
        except AssertionError as e:
            print(f"Test {i} failed: {e}, you had a better pokemon to choose!")


Choosing squirtle against geodude:
Test 1 passed, it was the best choice for the battle
Choosing squirtle against onix:
Test 2 passed, it was the best choice for the battle
Choosing totodile against umbreon:
Test 3 passed, it was the best choice for the battle
Choosing pikachu against murkrow:
Test 4 passed, it was the best choice for the battle
Choosing snorlax against gyarados:
Test 5 passed, it was the best choice for the battle
Choosing sandshrew against alakazam:
Test 6 passed, it was the best choice for the battle
Choosing bellossom against feraligatr:
Test 7 passed, it was the best choice for the battle
Choosing jigglypuff against dragonite:
Test 8 passed, it was the best choice for the battle
